In [ ]:
import os
from dotenv import load_dotenv

from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [ ]:
load_dotenv(override=True)

os.environ.get("OPENAI_API_KEY")

In [ ]:
file = "D:\Visual Code\RAG\pdf\entrecomp.pdf"
loader = PyPDFLoader(file)
documents = loader.load()

In [ ]:
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)

In [ ]:
embedding_model = OpenAIEmbeddings()

In [ ]:
directory = "../vector/chroma_vectorstore"

vector_store = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory=directory
)

In [ ]:
vector_store = Chroma(
    embedding_function=embedding_model,
    persist_directory=directory
)

In [ ]:
question = "O arquivo se trata de que?"

docs = vector_store.similarity_search(question, k=5)

In [ ]:
for doc in docs:
    print(doc.page_content)
    print(f"==== {doc.metadata}\n")